<a href="https://colab.research.google.com/github/Justus-coded/DPhi-Bootcamps/blob/master/Machine%20Learning%20Bootcamp/DPhi_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from statistics import mean 

import seaborn as sns
import matplotlib.pyplot as plt

from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV

In [2]:
loan_data  = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/Loan_Data/loan_train.csv" )

In [3]:
loan_data

,Unnamed: 0,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,LP002305,Female,No,0,Graduate,No,4547,0.0,115.0,360.0,1.0,Semiurban,1
1,1,LP001715,Male,Yes,3+,Not Graduate,Yes,5703,0.0,130.0,360.0,1.0,Rural,1
2,2,LP002086,Female,Yes,0,Graduate,No,4333,2451.0,110.0,360.0,1.0,Urban,0
3,3,LP001136,Male,Yes,0,Not Graduate,Yes,4695,0.0,96.0,NaN,1.0,Urban,1
4,4,LP002529,Male,Yes,2,Graduate,No,6700,1750.0,230.0,300.0,1.0,Semiurban,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,486,LP002103,NaN,Yes,1,Graduate,Yes,9833,1833.0,182.0,180.0,1.0,Urban,1
487,487,LP001790,Female,No,1,Graduate,No,3812,0.0,112.0,360.0,1.0,Rural,1
488,488,LP001401,Male,Yes,1,Graduate,No,14583,0.0,185.0,180.0,1.0,Rural,1
489,489,LP002893,Male,No,0,Graduate,No,1836,33837.0,90.0,360.0,1.0,Urban,0


In [7]:
loan_data.apply(lambda x: x.nunique())

Unnamed: 0           491
Loan_ID              491
Gender                 2
Married                2
Dependents             4
Education              2
Self_Employed          2
ApplicantIncome      408
CoapplicantIncome    239
LoanAmount           176
Loan_Amount_Term      10
Credit_History         2
Property_Area          3
Loan_Status            2
dtype: int64

In [8]:
loan = loan_data.drop(columns=['Unnamed: 0','Loan_ID'])

In [10]:
loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491 entries, 0 to 490
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             481 non-null    object 
 1   Married            490 non-null    object 
 2   Dependents         482 non-null    object 
 3   Education          491 non-null    object 
 4   Self_Employed      462 non-null    object 
 5   ApplicantIncome    491 non-null    int64  
 6   CoapplicantIncome  491 non-null    float64
 7   LoanAmount         475 non-null    float64
 8   Loan_Amount_Term   478 non-null    float64
 9   Credit_History     448 non-null    float64
 10  Property_Area      491 non-null    object 
 11  Loan_Status        491 non-null    int64  
dtypes: float64(4), int64(2), object(6)
memory usage: 46.2+ KB


In [11]:
loan.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status
count,491.000000,491.000000,475.000000,478.000000,448.000000,491.000000
mean,5401.189409,1589.730998,145.014737,341.297071,0.848214,0.698574
std,6419.427177,2919.320624,86.310534,66.964051,0.359214,0.459345
min,150.000000,0.000000,17.000000,12.000000,0.000000,0.000000
25%,2923.500000,0.000000,100.000000,360.000000,1.000000,0.000000
50%,3865.000000,1229.000000,126.000000,360.000000,1.000000,1.000000
75%,5705.500000,2251.500000,162.000000,360.000000,1.000000,1.000000
max,81000.000000,41667.000000,700.000000,480.000000,1.000000,1.000000


In [12]:
loan.Dependents = loan.Dependents.replace('3+', 3)

In [13]:
loan.Dependents.value_counts()

0    276
1     85
2     78
3     43
Name: Dependents, dtype: int64

In [39]:
loan1 = loan.dropna()

In [40]:
loan1.Dependents = loan1.Dependents.astype(int)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [41]:
loan1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 381 entries, 0 to 490
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             381 non-null    object 
 1   Married            381 non-null    object 
 2   Dependents         381 non-null    int64  
 3   Education          381 non-null    object 
 4   Self_Employed      381 non-null    object 
 5   ApplicantIncome    381 non-null    int64  
 6   CoapplicantIncome  381 non-null    float64
 7   LoanAmount         381 non-null    float64
 8   Loan_Amount_Term   381 non-null    float64
 9   Credit_History     381 non-null    float64
 10  Property_Area      381 non-null    object 
 11  Loan_Status        381 non-null    int64  
dtypes: float64(4), int64(3), object(5)
memory usage: 38.7+ KB


In [23]:
cat_cols = loan.select_dtypes(include = 'O')

In [43]:
loan2 = pd.get_dummies(loan1)

In [25]:
std = StandardScaler()

In [44]:
loan3 = std.fit_transform(loan2)

In [45]:
loan4 = pd.DataFrame(loan3)
loan4.columns = loan2.columns
loan4

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Gender_Female,Gender_Male,Married_No,Married_Yes,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,-0.775394,-0.128827,-0.618017,-0.344181,0.273943,0.397561,0.637106,2.107809,-2.107809,1.389521,-1.389521,0.490961,-0.490961,0.401977,-0.401977,-0.665719,1.214095,-0.637106
1,2.139986,0.069110,-0.618017,-0.155736,0.273943,0.397561,0.637106,-0.474426,0.474426,-0.719672,0.719672,-2.036823,2.036823,-2.487706,2.487706,1.502135,-0.823659,-0.637106
2,-0.775394,-0.165469,0.390262,-0.406997,0.273943,0.397561,-1.569598,2.107809,-2.107809,-0.719672,0.719672,0.490961,-0.490961,0.401977,-0.401977,-0.665719,-0.823659,1.569598
3,1.168193,0.239822,0.101888,1.100569,-0.636809,0.397561,0.637106,-0.474426,0.474426,-0.719672,0.719672,0.490961,-0.490961,0.401977,-0.401977,-0.665719,1.214095,-0.637106
4,-0.775394,-0.163586,-0.315246,-0.695947,0.273943,0.397561,-1.569598,-0.474426,0.474426,-0.719672,0.719672,-2.036823,2.036823,-2.487706,2.487706,-0.665719,1.214095,-0.637106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,-0.775394,-0.280704,1.465183,0.560358,0.273943,0.397561,0.637106,-0.474426,0.474426,1.389521,-1.389521,0.490961,-0.490961,0.401977,-0.401977,-0.665719,1.214095,-0.637106
377,0.196399,-0.254678,-0.618017,-0.381870,0.273943,0.397561,0.637106,2.107809,-2.107809,1.389521,-1.389521,0.490961,-0.490961,0.401977,-0.401977,1.502135,-0.823659,-0.637106
378,0.196399,1.589593,-0.618017,0.535232,-2.458314,0.397561,0.637106,-0.474426,0.474426,-0.719672,0.719672,0.490961,-0.490961,0.401977,-0.401977,1.502135,-0.823659,-0.637106
379,-0.775394,-0.593019,13.301659,-0.658258,0.273943,0.397561,-1.569598,-0.474426,0.474426,1.389521,-1.389521,0.490961,-0.490961,0.401977,-0.401977,-0.665719,-0.823659,1.569598


In [46]:
X = loan4.drop(['Loan_Status'], axis=1)
y = loan1['Loan_Status']

In [31]:
from sklearn.model_selection import train_test_split

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
from lightgbm import LGBMClassifier

In [34]:
xgb = XGBClassifier()
lgb = LGBMClassifier()

In [49]:
y.value_counts()

1    271
0    110
Name: Loan_Status, dtype: int64

In [50]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
f1_score(y_test, xgb_pred)

0.8709677419354839

In [51]:
lgb = LGBMClassifier()
lgb.fit(X_train, y_train)
lgb_pred = lgb.predict(X_test)
f1_score(y_test, lgb_pred)

0.8333333333333334

In [64]:
test_data = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/Loan_Data/loan_test.csv')

In [75]:
test_data.head(3)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001116,Male,No,0,Not Graduate,No,3748,1668.0,110.0,360.0,1.0,Semiurban
1,LP001488,Male,Yes,3+,Graduate,No,4000,7750.0,290.0,360.0,1.0,Semiurban
2,LP002138,Male,Yes,0,Graduate,No,2625,6250.0,187.0,360.0,1.0,Rural


In [76]:
test_data1 = test_data.copy()

In [77]:
test_data1.Gender.value_counts()

Male      96
Female    24
Name: Gender, dtype: int64

In [78]:
test_data1.Gender = test_data1.Gender.fillna('Male')

In [81]:
test_data1.Married.value_counts()

Yes    74
No     47
Name: Married, dtype: int64

In [82]:
test_data1.Married = test_data1.Married.fillna('Yes')


In [83]:
test_data1.Dependents.value_counts()

0     69
2     23
1     17
3+     8
Name: Dependents, dtype: int64

In [84]:
test_data1.Dependents = test_data1.Dependents.fillna(0)

In [85]:
test_data1.Self_Employed.value_counts()

No     102
Yes     18
Name: Self_Employed, dtype: int64

In [86]:
test_data1.Self_Employed= test_data1.Self_Employed.fillna('No')

In [87]:
test_data1.LoanAmount.describe()

count    117.000000
mean     152.085470
std       82.703623
min        9.000000
25%      104.000000
50%      131.000000
75%      180.000000
max      570.000000
Name: LoanAmount, dtype: float64

In [88]:
test_data1.LoanAmount = test_data1.LoanAmount.fillna(152)

In [90]:
test_data1.Loan_Amount_Term.describe()

count    122.000000
mean     344.754098
std       57.499021
min       60.000000
25%      360.000000
50%      360.000000
75%      360.000000
max      480.000000
Name: Loan_Amount_Term, dtype: float64

In [91]:
test_data1.Loan_Amount_Term = test_data1.Loan_Amount_Term.fillna(360)

In [92]:
test_data1.Credit_History.value_counts()

1.0    95
0.0    21
Name: Credit_History, dtype: int64

In [93]:
test_data1.Credit_History= test_data1.Credit_History.fillna(1.0)


In [94]:
test_data1.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

In [95]:
#test = test_data1.dropna()
test = test_data1.drop(['Loan_ID'], axis=1)

In [96]:
test.Dependents = test.Dependents.replace('3+', 3)

In [97]:
test.Dependents = test.Dependents.astype(int)

In [98]:
test1 = pd.get_dummies(test)

In [99]:
test2 = std.fit_transform(test1)
test2 = pd.DataFrame(test2)
test2.columns = test1.columns
test2.head(3)

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,-0.717710,-0.356265,-0.026793,-0.523948,0.265083,0.453743,-0.492366,0.492366,1.271622,-1.271622,-1.722714,1.722714,0.414039,-0.414039,-0.555437,1.271622,-0.786398
1,2.326372,-0.302328,2.034669,1.717205,0.265083,0.453743,-0.492366,0.492366,-0.786398,0.786398,0.580480,-0.580480,0.414039,-0.414039,-0.555437,1.271622,-0.786398
2,-0.717710,-0.596626,1.526252,0.434768,0.265083,0.453743,-0.492366,0.492366,-0.786398,0.786398,0.580480,-0.580480,0.414039,-0.414039,1.800383,-0.786398,-0.786398


In [100]:
test2.shape

(123, 17)

In [101]:
X_train.shape

(304, 17)

In [102]:
test_pred = xgb.predict(test2)
pd.Series(test_pred).to_csv('ass4.csv', index=False, header=['prediction'] )

In [103]:
pip install catboost

     |████████████████████████████████| 66.3MB 50kB/s 


In [104]:
from catboost import CatBoostClassifier

In [106]:
cat = CatBoostClassifier(task_type='GPU')

In [107]:
cat.fit(X_train, y_train, verbose=0)
cat_pred = cat.predict(X_test)
f1_score(y_test, cat_pred)

0.8617886178861788

In [108]:
test_pred1 = cat.predict(test2)
pd.Series(test_pred1).to_csv('ass5.csv', index=False, header=['prediction'] )